In [1]:
%%bash
# Buat folder cache di workspace (biar tidak makan storage utama 20GB)
mkdir -p /workspace/hf_cache

# Set environment variables permanen untuk session ini
echo 'export TRANSFORMERS_CACHE=/workspace/hf_cache' >> ~/.bashrc
echo 'export HF_HOME=/workspace/hf_cache' >> ~/.bashrc
echo 'export HF_HUB_CACHE=/workspace/hf_cache' >> ~/.bashrc

# Terapkan langsung ke session aktif
export TRANSFORMERS_CACHE=/workspace/hf_cache
export HF_HOME=/workspace/hf_cache
export HF_HUB_CACHE=/workspace/hf_cache

# Bersihkan cache lama yang makan ruang
rm -rf ~/.cache/huggingface
rm -rf ~/.cache/pip
echo "✅ Cache diarahkan ke /workspace/hf_cache"


✅ Cache diarahkan ke /workspace/hf_cache


In [2]:
!pip install transformers==4.38.2 accelerate==0.27.2 bitsandbytes==0.42.0 pydantic==2.9.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 1.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 2.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.3 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 3.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 1.1 MB/s eta 0:00:0000:0100:03m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 1.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 890.1 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 793.2 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import random

In [4]:
model_id = "deepseek-ai/deepseek-coder-6.7b-instruct"

print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading tokenizer...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype="auto",
    load_in_8bit=True,
)

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [7]:
from prompt_generator import CodeReviewPromptGenerator
prompt_generator = CodeReviewPromptGenerator()

In [21]:
import os
import json

datasets = os.listdir('.')

max_tokens_code_review_result = []
max_tokens_code_review_prompt = []
prompts = []
temp_dataset = []
for dataset in datasets:
    if not dataset.endswith('.json'):
        continue
    with open(f'{dataset}', 'r') as f:

        data = json.load(f)
        for i in data:
            max_tokens_code_review_result.append(len(i['code_review_suggestion']))
            prompt = prompt_generator.generate_style_review_prompt(
                added_code=i['added_code'],
                deleted_code=i['deleted_code'],
                full_function_code=i['full_function_code'],
                function_name=i['function_name'],
            )
            i['prompt'] = prompt
            temp_dataset.append(i)
            max_tokens_code_review_prompt.append(len(prompt))
            prompts.append(prompt)

avg_max_tokens_code_review_result = sum(max_tokens_code_review_result) / len(max_tokens_code_review_result) if max_tokens_code_review_result else 0
avg_max_tokens_code_review_prompt = sum(max_tokens_code_review_prompt) / len(max_tokens_code_review_prompt) if max_tokens_code_review_prompt else 0

In [11]:
print("avg_max_tokens_code_review_result:", avg_max_tokens_code_review_result)
print("avg_max_tokens_code_review_prompt:", avg_max_tokens_code_review_prompt)

avg_max_tokens_code_review_result: 461.07894736842104
avg_max_tokens_code_review_prompt: 1268.1368421052632


In [22]:
prompts = random.choices(temp_dataset, k=20)

In [23]:
prompts

[{'function_name': 'validate_user_input',
  'added_code': [{'start_line': 9,
    'end_line': 10,
    'code': '        logging.error("Validation error: " + str(e))',
    'line_count': 1}],
  'deleted_code': [{'start_line': 0,
    'end_line': 0,
    'code': '',
    'line_count': 0}],
  'full_function_code': 'def validate_user_input(data):\n    try:\n        if not isinstance(data, dict):\n            raise ValueError("Input must be a dictionary")\n        if "username" not in data or "password" not in data:\n            raise ValueError("Username and password are required")\n        if len(data["username"]).strip() == 0 or len(data["password"].strip()) == 0:\n            raise ValueError("Username and password cannot be empty")\n        print("Input validation successful")\n        return True\n    except ValueError as e:\n        print("Error: " + str(e))\n        logging.error("Validation error: " + str(e))\n        return False',
  'code_review_suggestion': 'SUMMARY: The code change a

In [28]:
results = []
for counter, prompt in enumerate(prompts):
    print("process {}/{}".format(counter+1, len(prompts)))
    inputs = tokenizer(prompt['prompt'], return_tensors="pt")
    with torch.inference_mode():
        outputs = model.generate(
            **inputs,
            max_new_tokens=300,
            temperature=0.1,
            top_k=20,
            top_p=0.8,
            do_sample=True,
            repetition_penalty=1.2,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

        raw_result = tokenizer.decode(outputs[0], skip_special_tokens=True)
        generated_tokens = outputs[0][inputs["input_ids"].shape[-1]:]  # skip prompt tokens
        text = tokenizer.decode(generated_tokens, skip_special_tokens=True)
        i['raw_result'] =  raw_result
        i['truncated_result'] = text
        results.append(i)

process 1/20


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1477: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


process 2/20
process 3/20
process 4/20
process 5/20
process 6/20
process 7/20
process 8/20
process 9/20
process 10/20
process 11/20
process 12/20
process 13/20
process 14/20
process 15/20
process 16/20
process 17/20
process 18/20
process 19/20
process 20/20


In [29]:
with open('result_before_fine_tunning.json', 'w') as f:
    json.dump(results, f, indent=4)